In [1]:
import import_ipynb

import easydict
 
import os
import copy

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm

from model import SRCNN
from Dataset import TrainDataset, EvalDataset
from utils import AverageMeter, calc_psnr


importing Jupyter notebook from model.ipynb
importing Jupyter notebook from Dataset.ipynb
importing Jupyter notebook from utils.ipynb


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
def train(train_file, eval_file, outputs_dir, scale,
         lr, batch_size, num_epochs, num_workers, seed):
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    if not os.path.exists(outputs_dir):
        os.makedirs(outputs_dir)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    
    model = SRCNN().to(device)
    
    criterion = nn.MSELoss()
    
    optimizer = optim.Adam([
        {'params': model.conv1.parameters()},
        {'params': model.conv2.parameters()},
        {'params': model.conv3.parameters(), 'lr': args.lr * 0.1}
    ], lr=args.lr)
    
    train_dataset = TrainDataset(train_file)
    train_dataloader = DataLoader(dataset=train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=num_workers,
                                  pin_memory=True,
                                  drop_last=True)
    
    eval_dataset = EvalDataset(eval_file)
    eval_dataloader = DataLoader(dataset=eval_dataset, batch_size=1)
    
    best_weights = copy.deepcopy(model.state_dict())
    best_epoch = 0
    best_psnr = 0.0
    
    for epoch in range(num_epochs):
        model.train()
        epoch_losses = AverageMeter()
        
        with tqdm(total=(len(train_dataset) - len(train_dataset) % batch_size)) as t:
            t.set_description('epoch: {}/{}'.format(epoch, num_epochs - 1))
            
            for data in train_dataloader:
                inputs, labels = data

                inputs = inputs.to(device)
                labels = labels.to(device)

                preds = model(inputs)
                loss = criterion(preds, labels)

                epoch_losses.update(loss.item(), len(inputs))

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                t.set_postfix(loss='{:.6f}'.format(epoch_losses.avg))
                t.update(len(inputs))

        torch.save(model.state_dict(), os.path.join(outputs_dir, 'epoch_{}.pth'.format(epoch)))
        
        model.eval()
        epoch_psnr = AverageMeter()

        for data in eval_dataloader:
            inputs, labels = data

            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                preds = model(inputs).clamp(0.0, 1.0)

            epoch_psnr.update(calc_psnr(preds, labels), len(inputs))

        print('eval psnr: {:.2f}'.format(epoch_psnr.avg))

        if epoch_psnr.avg > best_psnr:
            best_epoch = epoch
            best_psnr = epoch_psnr.avg
            best_weights = copy.deepcopy(model.state_dict())

    print('best epoch: {}, psnr: {:.2f}'.format(best_epoch, best_psnr))
    torch.save(best_weights, os.path.join(outputs_dir, 'best.pth'))


In [4]:
if __name__ == '__main__':
    
    args = easydict.EasyDict({
        "train_file": "/data/super-resolution/91_data/91-image_x3.h5",
        "eval_file": "/data/super-resolution/91_data/Set5_x2.h5",
        "outputs_dir": "/model/output/srcnn",
        "scale": 3,
        "lr": 1e-4,
        "batch_size": 16,
        "num_epochs": 400,
        "num_workers" : 8,
        "seed": 123,
})

    
    train(args.train_file, args.eval_file, args.outputs_dir, args.scale, args.lr,
         args.batch_size, args.num_epochs, args.num_workers, args.seed)
    
    

epoch: 1/399:   0%|          | 0/21872 [00:00<?, ?it/s]

eval psnr: 31.97


epoch: 2/399:   0%|          | 0/21872 [00:00<?, ?it/s]

eval psnr: 32.00


epoch: 3/399:   0%|          | 0/21872 [00:00<?, ?it/s]

...

eval psnr: 29.65


epoch: 399/399:   0%|          | 0/21872 [00:00<?, ?it/s]

eval psnr: 29.46


epoch: 399/399: 100%|██████████| 21872/21872 [00:17<00:00, 1240.23it/s, loss=0.001113]


eval psnr: 29.74
best epoch: 1, psnr: 32.00
